In [171]:
import pandas as pd
import spacy
import pickle
from gensim.models.ldamodel import LdaModel
from gensim import corpora

nlp = spacy.load("en_core_web_sm")

from absa_functions import *

In [2]:
train_path = 'train.xml'
corpus_df, categories = create_corpus(train_path)

In [3]:
reviews = corpus_df.text.to_list()
reviews[:5]

["It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.",
 'Hostess was extremely accommodating when we arrived an hour early for our reservation.',
 "We were a couple of minutes late for our reservation and minus one guest, but we didn't think we deserved the attitude we got from the hostess.",
 'Though the service might be a little slow, the waitresses are very friendly.',
 'Although we arrived at the restaurant 10 min late, the hostess did not have a table for us.']

In [108]:
#get syntactic dependecy relation, syntactic parent, coarse-grained part-of-speech tag for each token
count = 0
for review in reviews[:3]:
  count = 0
  doc = nlp(review)
  for token in doc:
    count += 1
    if count <= 10:
      print(token.text, token.dep_, token.head.text, token.head.pos_, token.pos_,[child for child in token.children])

It nsubj be VERB PRON []
might aux be VERB AUX []
be ROOT be VERB VERB [It, might, sit, ,, so, be]
the det best ADJ DET []
best nsubj sit VERB ADJ [the]
sit dep be VERB VERB [best, down, food]
down prt sit VERB ADP []
food dobj sit VERB NOUN [had]
I nsubj had VERB PRON []
've aux had VERB AUX []
Hostess nsubj accommodating VERB PROPN []
was aux accommodating VERB AUX []
extremely advmod accommodating VERB ADV []
accommodating ROOT accommodating VERB VERB [Hostess, was, extremely, arrived, .]
when advmod arrived VERB ADV []
we nsubj arrived VERB PRON []
arrived advcl accommodating VERB VERB [when, we, early]
an det hour NOUN DET []
hour npadvmod early ADV NOUN [an]
early advmod arrived VERB ADV [hour, for]
We nsubj were AUX PRON []
were ROOT were AUX AUX [We, couple, late, ,, but, think]
a det couple NOUN DET []
couple attr were AUX NOUN [a, of]
of prep couple NOUN ADP [minutes]
minutes pobj of ADP NOUN []
late advmod were AUX ADV [for]
for prep late ADV ADP [reservation]
our poss reser

In [6]:
#parse sentences for adjectives using POS tags
count = 0
for review in reviews:
    count += 1
    if count < 5:
        doc = nlp(review)
        adjectives = []
        for token in doc:
            if token.pos_ == 'ADJ':
                adjectives.append(token)
        print(review)
        print(adjectives)

It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.
[best, upright]
Hostess was extremely accommodating when we arrived an hour early for our reservation.
[]
We were a couple of minutes late for our reservation and minus one guest, but we didn't think we deserved the attitude we got from the hostess.
[]
Though the service might be a little slow, the waitresses are very friendly.
[little, slow, friendly]


In [7]:
#parse sentences for adverbs & adjectives using POS tags
count = 0
for review in reviews:
  count += 1
  if count <= 5:
    doc = nlp(review)
    adjectives = []
    for token in doc:
      if token.pos_ == 'ADJ':
        prepend = ''
        for child in token.children:
          if child.pos_ != 'ADV':
            continue
          prepend += child.text + ' '
        adjectives.append(prepend + token.text)
    print(review)
    print(adjectives)

It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.
['best', 'upright']
Hostess was extremely accommodating when we arrived an hour early for our reservation.
[]
We were a couple of minutes late for our reservation and minus one guest, but we didn't think we deserved the attitude we got from the hostess.
[]
Though the service might be a little slow, the waitresses are very friendly.
['little', 'slow', 'very friendly']
Although we arrived at the restaurant 10 min late, the hostess did not have a table for us.
[]


In [10]:
#check for subject nouns, assign to nearest adverbs & adjectives
count = 0
aspects = []
for review in reviews:
    count += 1
    if count <= 5:
        doc = nlp(review)
        descriptive_term = ''
        target = ''
        for token in doc:
            if (token.dep_ == 'nsubj' or token.dep_ == 'dobj') and token.pos_ == 'NOUN':
                target = token.text
            if token.pos_ == 'ADJ':
                prepend = ''
                for child in token.children:
                    if child.pos_ != 'ADV':
                        continue
                    prepend += child.text + ' '
                descriptive_term = prepend + token.text
        if target != '' and descriptive_term != '':
            aspects.append({'aspect': target,'description': descriptive_term})
print(aspects)

[{'aspect': 'food', 'description': 'upright'}, {'aspect': 'waitresses', 'description': 'very friendly'}]


In [ ]:
#check for subject nouns, assign to nearest adverbs & adjectives
count = 0
aspects = []
adjectives = []
targets = []
for review in reviews[:5]:
    doc = nlp(review)
    target = ''
    adjective = ''
    for token in doc:
        if (token.dep_ == 'nsubj' or token.dep_ == 'dobj') and token.pos_ == 'NOUN':
            target = token.text
            targets.append(target)
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            adjective = prepend + token.text
            adjectives.append(adjective)
        if target != '' and adjective != '':
            aspects.append({'aspect': target,'description': adjective})
print(aspects)

In [29]:
targets, adjectives

(['food', 'attitude', 'service', 'waitresses', 'hostess', 'table'],
 ['best', 'upright', 'little', 'slow', 'very friendly'])

In [157]:
def parse_targets(nlp, review):
    doc = nlp(review)
    targets = []
    target = ''

    for token in doc:
        if (token.dep_ in ['nsubj','dobj']) and (token.pos_ =='NOUN' or token.pos_ == 'PROPN'):
            target = token.text
            targets.append(target)

    return targets

In [158]:
def parse_adjectives(nlp, review):
    doc = nlp(review)
    adjectives = []
    adjective = ''

    for token in doc:
        if token.pos_ == 'ADJ' and token.dep_ != 'amod':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            adjective = prepend + token.text
            adjectives.append(adjective)

    return adjectives

In [153]:
reviews[1]

'Hostess was extremely accommodating when we arrived an hour early for our reservation.'

In [162]:
parse_adjectives(nlp, reviews[2])

[]

In [101]:
corpus_df.columns

Index(['text', 'sentiment', 'food', 'place', 'staff', 'miscellaneous',
       'service', 'price', 'menu', 'ambience'],
      dtype='object')

In [165]:
corpus_df['aspects'] = corpus_df['text'].apply(lambda x: parse_targets(nlp, x))
corpus_df['descriptions'] = corpus_df['text'].apply(lambda x: parse_adjectives(nlp, x))

In [166]:
corpus_df.head(15)

,text,sentiment,food,place,staff,miscellaneous,service,price,menu,ambience,aspects,descriptions
0,It might be the best sit down food I've had in...,"{'food': 'positive', 'place': 'neutral'}",positive,neutral,None,None,None,None,None,None,[food],[best]
1,Hostess was extremely accommodating when we ar...,"{'staff': 'positive', 'miscellaneous': 'neutral'}",None,None,positive,neutral,None,None,None,None,[Hostess],[]
2,We were a couple of minutes late for our reser...,"{'miscellaneous': 'neutral', 'staff': 'negative'}",None,None,negative,neutral,None,None,None,None,[attitude],[]
3,"Though the service might be a little slow, the...","{'service': 'negative', 'staff': 'positive'}",None,None,positive,None,negative,None,None,None,"[service, waitresses]","[slow, very friendly]"
4,Although we arrived at the restaurant 10 min l...,"{'staff': 'negative', 'miscellaneous': 'neutral'}",None,None,negative,neutral,None,None,None,None,"[hostess, table]",[]
5,I like the smaller portion size for dinner.,"{'miscellaneous': 'negative', 'food': 'neutral'}",neutral,None,None,negative,None,None,None,None,[size],[]
6,The bill was surprisingly inexpensive consider...,"{'food': 'neutral', 'price': 'positive', 'misc...",neutral,None,None,neutral,None,positive,None,None,"[bill, appetizers, rounds]","[surprisingly inexpensive, alcoholic, non]"
7,") other food is served in too-small portions, ...","{'miscellaneous': 'negative', 'food': 'neutral'}",neutral,None,None,negative,None,None,None,None,[room],[least]
8,"It was very loud, I felt too crowded, the man ...","{'miscellaneous': 'neutral', 'staff': 'negative'}",None,None,negative,neutral,None,None,None,None,"[chair, waiters]","[very loud, too crowded, next, impossible]"
9,"After ordering drinks, we both decided on the ...","{'food': 'neutral', 'miscellaneous': 'positive'}",neutral,None,None,positive,None,None,None,None,[drinks],[]


In [172]:
corpus = pickle.load(open('corpus.pkl', 'rb'))
dictionary = corpora.Dictionary.load('dictionary.gensim')
lda_model = LdaModel.load('best_lda_model.gensim')

In [175]:
corpus_df['aspects'][0]

['food']

In [190]:
#numerical mapping for topics in LDA model
topic_map = {0: 'menu', 1: 'service', 2: 'miscellaneous', 3: 'place', 4: 'price', 5: 'food', 6: 'staff'}

In [189]:
lda_model.get_term_topics('food', minimum_probability=0.0000001)

[(0, 1.079433e-06),
 (1, 5.546354e-07),
 (2, 7.8215083e-07),
 (3, 0.00036240902),
 (4, 0.00035394274),
 (5, 0.0011942886),
 (6, 4.8887154e-07)]